In [80]:
import time
import selenium
import requests
import io
import os
import time 
from PIL import Image
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import quote_plus  




In [81]:
def get_google_images_url(query):
    # Encode the query to ensure it's URL-safe
    encoded_query = quote_plus(query)
    return f"https://www.google.com/search?hl=en&tbm=isch&q={encoded_query}"

In [82]:
def get_imgs_from_google(driver,max_num_imgs,url):
    driver.get(url)
    img_urls=set()
    l = driver.find_element("id",'W0wltc')
    l.click()
    # imgs = driver.find_elements(By.CLASS_NAME, "mNsIhb")
    while len(img_urls) < max_num_imgs:
        imgs = driver.find_elements(By.CLASS_NAME, "mNsIhb")
        for i,img in enumerate(imgs[len(img_urls):], start=len(img_urls)):
            actions = ActionChains(driver)
            actions.move_to_element(img).perform()
            time.sleep(2)  # Wait for overlay elements to load
            driver.execute_script("arguments[0].click();", img)
            try:
                images = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, ".sFlh5c.FyHeAf.iPVvYb"))
                    )
            except Exception as e:
                print(f"{i}Error clicking img: {e}")
                continue
            img_url = images.get_attribute("src")
            img_urls.add(img_url)
            print(f"{i}/{max_num_imgs}-->{img_url}")
            if len(img_urls) >= max_num_imgs:
                print(f"Collected {max_num_imgs} images. Exiting loop.")
                break  
    driver.quit()
    return img_urls




In [94]:
def download_image (download_path, urls,prefix):
    if not os.path.exists(download_path):
        os.makedirs(download_path)  # Creates the directory if it doesn't exist
    for i , url in enumerate(urls):
        try:
            image_content = requests.get(url).content
            image_file = io.BytesIO(image_content)
            image = Image.open(image_file)
            file_path = os.path.join(download_path, f"{prefix}_{i + 1}.jpg")
                        # Convert image to RGB if it's in RGBA or P mode
            if image.mode in ("RGBA", "P"):
                image = image.convert("RGB")
            with open(file_path,"wb") as f :
                image.save(f, "JPEG")
        except Exception as e:
            print("FAILED -",e)

In [95]:
driver = webdriver.Chrome()  
search ="robots pictures"
url_robots = get_google_images_url(search)
urls_robots = get_imgs_from_google(driver,3,url_robots)
download_image("images//Robots",urls_robots,"robot")

0/3-->https://www.rolandberger.com/img/og/Roland_Berger-24_2195_Humanoid_robots-OG.jpg
1/3-->https://www.orientsoftware.com/Themes/Content/Images/blog/2022-01-07/robots-in-everyday-life.png
2/3-->https://media.licdn.com/dms/image/D4D12AQH1pZCGpcCr7w/article-cover_image-shrink_720_1280/0/1677021806463?e=2147483647&v=beta&t=XKkiYCvkNsBKpe0zf86t4WM0dxGQWZOGgO30UTj4LZU
Collected 3 images. Exiting loop.


In [ ]:
search ="people photography"
url_humans = get_google_images_url(search)
urls_humans = get_imgs_from_google(driver,200,url_humans)
download_image("images//Humans",urls_humans,"human")

success
success
success
success
success
